In [5]:
%matplotlib inline

In [6]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats
import scipy.io.wavfile
from scipy.fftpack import fft, fftfreq

# Error-Correcting Codes

## What are Error-Correcting codes and why are they so important: 

In the era of communications, it is always important the data to be transferred in a secure and looseless way. But we still cannot make the technologies so perfect, and sometime times part of the data can be lost during the transfer. 
So what can we do in order to check whether there are errors in data transmission over noisy or unreliable communications channels?
For example, we can generate a hashcode of the transferred data (using MD5, SHA256 or some other algorithm), and provide it together with the transferred data. If there is difference between the generated hashcode, and the hashcode, that we calculate at the final point of the transfer, we shall know, that there is difference between the original data, and the data that we received (some transfer loss, some intended change in the data being transferred).
If the content of the data is not so important for us, we can even just make a checksum - for example summing the numbers of the bytes to be transferred, and this sum will be transferred together with the data.  After the data is transferred, the checksum is calculated again, and if it is the same as the provided one, then we consider, that the transferred data is intact.
Ok, until now we found two easy ways to find that there was an error/problem during the data transmission. But what is the biggest problem with them?
The biggest problem is, that even if we find out, that a problem occurred during the data transfer, we can only detect it, but we CANNOT FIX it. And here come the error-correcting codes. 